In [59]:
import os

import pandas as pd
import mlflow
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    OneHotEncoder, 
    SplineTransformer, 
    QuantileTransformer, 
    RobustScaler,
    PolynomialFeatures,
    KBinsDiscretizer,
)

TABLE_NAME = "users_churn"

TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

EXPERIMENT_NAME = "krosh_16_07_2024"
RUN_NAME = "preprocessing" 
REGISTRY_MODEL_NAME = 'churn_model_krosh_b2c'

In [60]:
import psycopg
from dotenv import load_dotenv

load_dotenv()

connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": os.getenv("DB_DESTINATION_HOST"),
    "port": os.getenv("DB_DESTINATION_PORT"),
    "dbname": os.getenv("DB_DESTINATION_NAME"),
    "user": os.getenv("DB_DESTINATION_USER"),
    "password": os.getenv("DB_DESTINATION_PASSWORD"),
}

connection.update(postgres_credentials)

with psycopg.connect(**connection) as conn:

    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
        data = cur.fetchall()
        columns = [col[0] for col in cur.description]

df = pd.DataFrame(data, columns=columns)

df.head(5)

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
0,17,8191-XWSZG,2015-10-01,NaT,One year,No,Mailed check,20.65,1022.95,None,None,None,None,None,None,None,Female,0,No,No,No,0
1,59,3957-SQXML,2017-04-01,NaT,Two year,No,Credit card (automatic),24.95,894.30,None,None,None,None,None,None,None,Female,0,Yes,Yes,Yes,0
2,147,6837-BJYDQ,2019-11-01,NaT,One year,No,Mailed check,19.60,61.35,None,None,None,None,None,None,None,Male,0,No,No,No,0
3,482,0486-LGCCH,2019-03-01,NaT,Two year,No,Mailed check,19.65,225.75,None,None,None,None,None,None,None,Male,0,Yes,Yes,No,0
4,1,7590-VHVEG,2020-01-01,NaT,Month-to-month,Yes,Electronic check,29.85,29.85,DSL,No,Yes,No,No,No,No,Female,0,Yes,No,None,0


In [61]:
# Выделем колонки, которые содержат строковый формат
obj_df = df.select_dtypes(include="object")
print(obj_df)

     customer_id            type paperless_billing             payment_method   
0     8191-XWSZG        One year                No               Mailed check  \
1     3957-SQXML        Two year                No    Credit card (automatic)   
2     6837-BJYDQ        One year                No               Mailed check   
3     0486-LGCCH        Two year                No               Mailed check   
4     7590-VHVEG  Month-to-month               Yes           Electronic check   
...          ...             ...               ...                        ...   
7038  2823-LKABH  Month-to-month               Yes  Bank transfer (automatic)   
7039  8775-CEBBJ  Month-to-month               Yes  Bank transfer (automatic)   
7040  0550-DCXLH  Month-to-month                No               Mailed check   
7041  9281-CEDRU        Two year                No  Bank transfer (automatic)   
7042  2235-DWLJU  Month-to-month               Yes           Electronic check   

     internet_service onlin

In [62]:
# определение категориальных колонок, которые будут преобразованы
cat_columns = ["type", "payment_method", "internet_service", "gender"]

# создание объекта OneHotEncoder для преобразования категориальных переменных
# auto - автоматическое определение категорий
# ignore - игнорировать ошибки, если встречается неизвестная категория
# max_categories - максимальное количество уникальных категорий
# sparse_output - вывод в виде разреженной матрицы, если False, то в виде обычного массива
# drop="first" - удаляет первую категорию, чтобы избежать ловушки мультиколлинеарности
encoder_oh = OneHotEncoder(
    categories='auto',
    handle_unknown='ignore',
    max_categories=10,
    sparse_output=False,
    drop='first'
    )

for col in cat_columns:
    unique_values = df[col].unique()
    print(f"Уникальные значения в столбце '{col}': {unique_values}")

# применение OneHotEncoder к данным. Преобразование категориальных данных в массив
encoded_features = encoder_oh.fit_transform(df[cat_columns].to_numpy())

# преобразование полученных признаков в DataFrame и установка названий колонок
# get_feature_names_out() - получение имён признаков после преобразования
encoded_df = pd.DataFrame(encoded_features, columns=encoder_oh.get_feature_names_out(cat_columns))
print(encoded_df.head(5))
# конкатенация исходного DataFrame с новым DataFrame, содержащим закодированные категориальные признаки
# axis=1 означает конкатенацию по колонкам
obj_df = pd.concat([obj_df, encoded_df], axis=1)


Уникальные значения в столбце 'type': ['One year' 'Two year' 'Month-to-month']
Уникальные значения в столбце 'payment_method': ['Mailed check' 'Credit card (automatic)' 'Electronic check'
 'Bank transfer (automatic)']
Уникальные значения в столбце 'internet_service': [None 'DSL' 'Fiber optic']
Уникальные значения в столбце 'gender': ['Female' 'Male']
   type_One year  type_Two year  payment_method_Credit card (automatic)   
0            1.0            0.0                                     0.0  \
1            0.0            1.0                                     1.0   
2            1.0            0.0                                     0.0   
3            0.0            1.0                                     0.0   
4            0.0            0.0                                     0.0   

   payment_method_Electronic check  payment_method_Mailed check   
0                              0.0                          1.0  \
1                              0.0                          0.

In [63]:
# Преобразовываем числовые признаки с помощью пяти энкодеров (Спринт 2, Тема 3, Урок 5, Задание 2)
pd.set_option('display.max_columns', None)

num_columns = ["monthly_charges", "total_charges"]

num_df = df[num_columns]

n_knots = 3
degree_spline = 4
n_quantiles=100
degree = 3
n_bins = 5
encode = 'ordinal'
strategy = 'uniform'
subsample = None

print(df.shape[0])
# Энкодеры не работают с пропущенными значеничми, их нужно удалить.
if df[num_columns].isnull().values.any():
    print("Есть пропущенные значения в числовых столбцах")
    df = df.dropna(subset=num_columns)
print(df.shape[0])

# SplineTransformer
encoder_spl = SplineTransformer(n_knots=n_knots, degree=degree_spline)
encoded_features = encoder_spl.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(
    encoded_features, 
    columns=encoder_spl.get_feature_names_out(num_columns)
)
num_df = pd.concat([num_df, encoded_df], axis=1)


# QuantileTransformer
encoder_q = QuantileTransformer(n_quantiles=n_quantiles)
encoded_features = encoder_q.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(
    encoded_features, 
    columns=encoder_q.get_feature_names_out(num_columns)
)
encoded_df.columns = [col + f"_q_{n_quantiles}" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)

# RobustScaler
encoder_rb = RobustScaler()
encoded_features = encoder_rb.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(
    encoded_features, 
    columns=encoder_rb.get_feature_names_out(num_columns)
)
encoded_df.columns = [col + f"_robust" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)

# PolynomialFeatures
encoder_pol = PolynomialFeatures(degree=degree)
encoded_features = encoder_pol.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(
    encoded_features, 
    columns=encoder_pol.get_feature_names_out(num_columns)
)
encoded_df = encoded_df.drop(columns=num_columns)
num_df = pd.concat([num_df, encoded_df], axis=1)

# KBinsDiscretizer
encoder_kbd = KBinsDiscretizer(n_bins=n_bins, encode=encode, strategy=strategy, subsample=subsample)
encoded_features = encoder_kbd.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(
    encoded_features, 
    columns=encoder_kbd.get_feature_names_out(num_columns)
)
encoded_df.columns = [col + f"_bin" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)

num_df.head(5)

7043
Есть пропущенные значения в числовых столбцах
7032


,monthly_charges,total_charges,monthly_charges_sp_0,monthly_charges_sp_1,monthly_charges_sp_2,monthly_charges_sp_3,monthly_charges_sp_4,monthly_charges_sp_5,total_charges_sp_0,total_charges_sp_1,total_charges_sp_2,total_charges_sp_3,total_charges_sp_4,total_charges_sp_5,monthly_charges_q_100,total_charges_q_100,monthly_charges_robust,total_charges_robust,1,monthly_charges^2,monthly_charges total_charges,total_charges^2,monthly_charges^3,monthly_charges^2 total_charges,monthly_charges total_charges^2,total_charges^3,monthly_charges_bin,total_charges_bin
0,20.65,1022.95,0.034259,0.433936,0.481590,0.050214,2.168151e-07,0.0,0.014515,0.334777,0.555277,0.095311,1.201789e-04,0.0,0.151515,0.416558,-0.915707,-0.110372,1.0,426.4225,21123.9175,1.046427e+06,8805.624625,436208.896375,2.160871e+07,1.070442e+09,0.0,0.0
1,24.95,894.30,0.023507,0.388355,0.519449,0.068676,1.316872e-05,0.0,0.016892,0.350947,0.545446,0.086646,6.944782e-05,0.0,0.196970,0.387855,-0.836481,-0.148285,1.0,622.5025,22312.7850,7.997725e+05,15531.437375,556703.985750,1.995432e+07,7.152365e+08,0.0,0.0
2,19.60,61.35,0.037366,0.444730,0.471576,0.046328,2.170599e-08,0.0,0.040054,0.453400,0.463219,0.043328,3.874648e-10,0.0,0.040404,0.064566,-0.935053,-0.393755,1.0,384.1600,1202.4600,3.763823e+03,7529.536000,23568.216000,7.377092e+04,2.309105e+05,0.0,0.0
3,19.65,225.75,0.037214,0.444220,0.472059,0.046508,2.510479e-08,0.0,0.034259,0.433936,0.481591,0.050214,2.168182e-07,0.0,0.050505,0.179825,-0.934132,-0.345307,1.0,386.1225,4435.9875,5.096306e+04,7587.307125,87167.154375,1.001424e+06,1.150491e+07,0.0,0.0
4,29.85,29.85,0.014583,0.335266,0.554993,0.095040,1.183250e-04,0.0,0.041243,0.457057,0.459607,0.042093,1.762313e-12,0.0,0.232496,0.027400,-0.746200,-0.403038,1.0,891.0225,891.0225,8.910225e+02,26597.021625,26597.021625,2.659702e+04,2.659702e+04,0.0,0.0


In [64]:
# Объединим преобразования над числовыми колонками в ColumnTransformer, а над категориальными — в Pipeline (Спринт 2, Тема 3, Урок 5, Задание 3)
numeric_transformer = ColumnTransformer(
    transformers=[
        ('spl', encoder_spl, num_columns),
        ('q', encoder_q, num_columns),
        ('rb', encoder_rb, num_columns),
        ('pol', encoder_pol, num_columns),
        ('kbd', encoder_kbd, num_columns)
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ('encoder', encoder_oh)
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_columns),
        ('cat', categorical_transformer, cat_columns)
        ], 
    n_jobs=-1)

print(df.shape[0], df.shape[1])
encoded_features = preprocessor.fit_transform(df)

transformed_df = pd.DataFrame(encoded_features, columns=preprocessor.get_feature_names_out())

df = pd.concat([df, transformed_df], axis=1)
print(df.shape[0], df.shape[1])
df.head(2)

7032 22
7043 58


,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target,num__spl__monthly_charges_sp_0,num__spl__monthly_charges_sp_1,num__spl__monthly_charges_sp_2,num__spl__monthly_charges_sp_3,num__spl__monthly_charges_sp_4,num__spl__monthly_charges_sp_5,num__spl__total_charges_sp_0,num__spl__total_charges_sp_1,num__spl__total_charges_sp_2,num__spl__total_charges_sp_3,num__spl__total_charges_sp_4,num__spl__total_charges_sp_5,num__q__monthly_charges,num__q__total_charges,num__rb__monthly_charges,num__rb__total_charges,num__pol__1,num__pol__monthly_charges,num__pol__total_charges,num__pol__monthly_charges^2,num__pol__monthly_charges total_charges,num__pol__total_charges^2,num__pol__monthly_charges^3,num__pol__monthly_charges^2 total_charges,num__pol__monthly_charges total_charges^2,num__pol__total_charges^3,num__kbd__monthly_charges,num__kbd__total_charges,cat__type_One year,cat__type_Two year,cat__payment_method_Credit card (automatic),cat__payment_method_Electronic check,cat__payment_method_Mailed check,cat__internet_service_Fiber optic,cat__internet_service_None,cat__gender_Male
0,17.0,8191-XWSZG,2015-10-01,NaT,One year,No,Mailed check,20.65,1022.95,None,None,None,None,None,None,None,Female,0.0,No,No,No,0.0,0.034259,0.433936,0.481590,0.050214,2.168151e-07,0.0,0.014515,0.334777,0.555277,0.095311,0.000120,0.0,0.151515,0.416558,-0.915707,-0.110372,1.0,20.65,1022.95,426.4225,21123.9175,1.046427e+06,8805.624625,436208.896375,2.160871e+07,1.070442e+09,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,59.0,3957-SQXML,2017-04-01,NaT,Two year,No,Credit card (automatic),24.95,894.30,None,None,None,None,None,None,None,Female,0.0,Yes,Yes,Yes,0.0,0.023507,0.388355,0.519449,0.068676,1.316872e-05,0.0,0.016892,0.350947,0.545446,0.086646,0.000069,0.0,0.196970,0.387855,-0.836481,-0.148285,1.0,24.95,894.30,622.5025,22312.7850,7.997725e+05,15531.437375,556703.985750,1.995432e+07,7.152365e+08,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0


In [65]:
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net"
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY")

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id

    mlflow.sklearn.log_model(preprocessor, "column_transformer")
    
# Возникает ошибка в логировании preprocessor, так как в нем метода predict!

2024/07/20 10:49:18 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


In [66]:
print(run_id)

e0c5b3be381641c2a58a3e2ef7e3f0b1
